# GPT From Scratch

Based of the video tutorial from Andrej Karpathy [Link](https://www.youtube.com/watch?v=kCc8FmEb1nY&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)

In [3]:
# Load the dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O ../datasets/tinyshakespeare/input.txt

--2024-04-08 13:07:00--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘../datasets/tinyshakespeare/input.txt’

../datasets/tinysha 100%[===================>]   1.06M  2.29MB/s    in 0.5s    

2024-04-08 13:07:00 (2.29 MB/s) - ‘../datasets/tinyshakespeare/input.txt’ saved [1115394/1115394]



In [10]:
# Install dependencies
!pip install torch torchvision

  Using cached torch-2.2.2-cp312-none-macosx_11_0_arm64.whl.metadata (25 kB)
  Using cached torchvision-0.17.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached filelock-3.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pillow-10.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.2.2-cp312-none-macosx_11_0_arm64.whl (59.7 MB)
Using cached torchvision-0.17.2-cp312-cp312-macosx_11_0_arm64.whl (1.6 MB)
Using cached pillow-10.3.0-cp312-cp312-macosx_11_0_arm64.whl (3.4 MB)
Using cached filelock-3.13.3-py3-none-any.whl (11 kB)
Usi

In [ ]:
# Import dependencies

import torch


In [ ]:
with open('../datasets/tinyshakespeare/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print (f"Lenght of text: {len(text)}")

print (text[:1000])

In [8]:
# Generate tokens (on a character level)
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
# Next lets generate the character to integer encoding
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l]) 

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there
